## TF IDF & MLP

Here we are given two articles and we have to check similarity between them,
There are many metrics of similarity, <br>
<br>
One of them is TF IDF ( Total Frequency * Inverse Document Frequency). In this metric we multiply the number of times a word is present in a document to the inverse of the number of the articles that the word appears in. The inverse part shows the overall importance of the word and the frequency part shows how important the given word is in the context of the present article. <br>
<br>


###Loading the data

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
import json
f = open('/content/drive/MyDrive/Colab Notebooks/SMAI Project/train_data_multilable.json')
train_data = json.load(f)
g= open('/content/drive/MyDrive/Colab Notebooks/SMAI Project/test_data_multilable.json')
test_data = json.load(g)

In [41]:
print(test_data[0].keys())

print(len(test_data))

dict_keys(['pair_id', 'title_1', 'title_2', 'text_1', 'text_2', 'tags_1', 'tags_2', 'meta_lang_1', 'meta_lang_2', 'meta_description_1', 'meta_description_2', 'score', 'score_1', 'score_2', 'score_3', 'score_4', 'score_5', 'score_6'])
4316


In [42]:
print(train_data[0].keys())

print(len(train_data))

dict_keys(['pair_id', 'title_1', 'title_2', 'text_1', 'text_2', 'tags_1', 'tags_2', 'meta_lang_1', 'meta_lang_2', 'score', 'score_1', 'score_2', 'score_3', 'score_4', 'score_5', 'score_6'])
2857


After Filtering out the english articles - we compare the two articles in a given element. 

For that we append the two articles with keys : text_1 and text_2 

In [43]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from string import punctuation


english_articles_train = []
english_articles_test = []


for d in train_data:
  if d['meta_lang_1'] == 'en' and d['meta_lang_2'] == 'en':
    english_articles_train.append(d)

for d in test_data:
  if d['meta_lang_1'] == 'en' and d['meta_lang_2'] == 'en':
    english_articles_test.append(d)



total_train_articles = []
total_test_articles = []

y_train = []
y_test = []

for eng in english_articles_train:
  total_train_articles.append(eng['text_1'])
  total_train_articles.append(eng['text_2'])
  y = []
  y.append(eng['score'])
  y.append(eng['score_1'])
  y.append(eng['score_2'])
  y.append(eng['score_3'])
  y.append(eng['score_4'])
  y.append(eng['score_5'])
  y.append(eng['score_6'])
  y_train.append(np.array(y))
  

for eng in english_articles_test:
  total_test_articles.append(eng['text_1'])
  total_test_articles.append(eng['text_2'])
  y = []
  y.append(eng['score'])
  y.append(eng['score_1'])
  y.append(eng['score_2'])
  y.append(eng['score_3'])
  y.append(eng['score_4'])
  y.append(eng['score_5'])
  y.append(eng['score_6'])
  y_test.append(np.array(y))


'''
art1 = art1.split()

art2 = art2.split()

for i in range(len(art1)):
    art1[i] = ''.join([letter for letter in art1[i] if letter not in punctuation])

for i in range(len(art2)):
    art2[i] = ''.join([letter for letter in art2[i] if letter not in punctuation])
'''

#########################################################



tf = TfidfVectorizer(min_df =2)

x_train = tf.fit(total_train_articles)
x_train = tf.transform(total_train_articles)
x_test = tf.transform(total_test_articles)


print(x_train.shape)

print(type(x_train))



X_text_1_train = []
X_text_2_train = []
X_text_1_test = []
X_text_2_test = []


print(x_test.shape)
print(type(x_test))



for j in range(1287):
  X_text_1_train.append(x_train[2*j])
  X_text_2_train.append(x_train[2*j+1])

for j in range(235):
  X_text_1_test.append(x_test[2*j])
  X_text_2_test.append(x_test[2*j+1])


(2574, 25972)
<class 'scipy.sparse.csr.csr_matrix'>
(470, 25972)
<class 'scipy.sparse.csr.csr_matrix'>


In [44]:
from scipy.sparse import csr_matrix
 

X1_train = []
X2_train = []
X1_test = []
X2_test = []

for i in range(1287):
  X1_train.append(np.array(csr_matrix.todense(X_text_1_train[i])))
  X2_train.append(np.array(csr_matrix.todense(X_text_2_train[i])))

for i in range(235):
  X1_test.append(np.array(csr_matrix.todense(X_text_1_test[i])))
  X2_test.append(np.array(csr_matrix.todense(X_text_2_test[i])))


y_train = np.array(y_train)
print(y_train.shape)
y_test = np.array(y_test)

#scor1 = np.reshape(scor1,(1287,1))
X1_train = np.array(X1_train)
X2_train = np.array(X2_train)
X1_test = np.array(X1_test)
X2_test = np.array(X2_test)

#rows1 = np.reshape()

X1_train = X1_train.reshape(1287,25972)
X2_train = X2_train.reshape(1287,25972)
X1_test = X1_test.reshape(235,25972)
X2_test = X2_test.reshape(235,25972)



(1287, 7)


In [45]:
print(X1_train.shape)

(1287, 25972)


#Model

This is the Siamese implementation of the 

In [46]:
from tensorflow import keras
from keras.models import Model
from keras.layers import Dense, LSTM, Input, concatenate

input_1 = Input(shape=(25972, ))
dense_1 = Dense(1000, )(input_1)

input_2 = Input(shape=(25972, ))
dense_2 = Dense(1000, )(input_2)

merge = concatenate([dense_1, dense_2])

dense_3  = Dense(200,)(merge)
dense_4  = Dense(50,)(dense_3)
dense_all = Dense(7,)(dense_4)
#second_dense = Dense(2, activation='softmax')(merge)

model = Model(inputs=[input_1, input_2], outputs=dense_all)
optimizer = keras.optimizers.Adam()  #learning_rate=0.001
model.compile(loss='mse', optimizer = optimizer) #, metrics=['RootMeanSquaredError']
print(model.summary())

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 25972)]      0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, 25972)]      0           []                               
                                                                                                  
 dense_15 (Dense)               (None, 1000)         25973000    ['input_7[0][0]']                
                                                                                                  
 dense_16 (Dense)               (None, 1000)         25973000    ['input_8[0][0]']                
                                                                                            

In [47]:
history = model.fit([X1_train, X2_train], y_train, epochs=35, validation_split= 0.1)

Epoch 1/35
37/37 [==============================] - 1s 20ms/step - loss: 1.6945 - val_loss: 1.5505
Epoch 2/35
37/37 [==============================] - 1s 16ms/step - loss: 0.4747 - val_loss: 1.6559
Epoch 3/35
37/37 [==============================] - 1s 17ms/step - loss: 0.1753 - val_loss: 1.5841
Epoch 4/35
37/37 [==============================] - 1s 16ms/step - loss: 0.0828 - val_loss: 1.5813
Epoch 5/35
37/37 [==============================] - 1s 17ms/step - loss: 0.0528 - val_loss: 1.5772
Epoch 6/35
37/37 [==============================] - 1s 15ms/step - loss: 0.0446 - val_loss: 1.6036
Epoch 7/35
37/37 [==============================] - 1s 15ms/step - loss: 0.0364 - val_loss: 1.5507
Epoch 8/35
37/37 [==============================] - 1s 16ms/step - loss: 0.0398 - val_loss: 1.5906
Epoch 9/35
37/37 [==============================] - 1s 16ms/step - loss: 0.0345 - val_loss: 1.5832
Epoch 10/35
37/37 [==============================] - 1s 15ms/step - loss: 0.0343 - val_loss: 1.5769
Epoch 11/

## Test Metrics

In [48]:
y_test_pred = model.predict([X1_test,X2_test])

8/8 [==============================] - 0s 3ms/step


In [49]:
print(y_test_pred.shape)

(235, 7)


In [50]:
print(y_test.shape)

(235, 7)


In [51]:
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.feature_selection import r_regression
print('The mean absolute error for the model is ', mean_absolute_error(y_test[0], y_test_pred[0]))
print('The mean squared error for the model is ', mean_squared_error(y_test[0], y_test_pred[0]))

The mean absolute error for the model is  0.4751460296767099
The mean squared error for the model is  0.5113956187010551


In [52]:
from scipy.stats import pearsonr

print(pearsonr(y_test[0],y_test_pred[0]))

(0.5683369961342529, 0.18311410956253396)


## Train Metrics

In [53]:
y_train_pred = model.predict([X1_train,X2_train])

41/41 [==============================] - 0s 3ms/step


In [54]:
print(y_train_pred.shape)

(1287, 7)


In [55]:
print(y_train.shape)

(1287, 7)


In [56]:
from scipy.stats import pearsonr

print(pearsonr(y_train[0],y_train_pred[0]))

(0.9996764905366035, 3.6148979557072408e-09)
